In [95]:
from pyzabbix import ZabbixAPI
import pandas as pd
from requests.auth import HTTPBasicAuth
import requests
import re
import pytz
import datetime
import time
from dateutil.relativedelta import relativedelta
#from datetime import timedelta

In [7]:
zapi = ZabbixAPI("http://10.10.10.20/zabbix")
#zapi.login("Admin", "zabbix")

## total server/VM

In [102]:
get_hosts = zapi.host.get(url = "http://10.10.10.20/zabbix")

#get host_id
df_host = pd.DataFrame(get_hosts)
df_host = df_host.filter(items=['hostid', 'host','name']).drop_duplicates()
print('number of server/VM :' , len(df_host))
# df_host

# lst_host_id = list(df_host['hostid'])
# lst_host_id

# lst_host_name = list(df_host['name'])
# lst_host_name

number of server/VM : 40


## Find item id

In [103]:
get_items = zapi.item.get(url = "http://10.10.10.20/zabbix")

# get item data
df_item= pd.DataFrame(get_items)
df_item

,itemid,type,snmp_oid,hostid,name,key_,delay,history,trends,status,...,allow_traps,discover,uuid,state,error,parameters,lastclock,lastns,lastvalue,prevvalue
0,10020,0,,10001,Zabbix agent ping,agent.ping,1m,7d,365d,0,...,0,0,,0,,[],0,0,0,0
1,10059,0,,10001,Version of Zabbix agent running,agent.version,1h,7d,0,0,...,0,0,,0,,[],0,0,,
2,10061,5,,10047,Zabbix server: Number of processed numeric (fl...,"zabbix[wcache,values,float]",1m,1w,365d,0,...,0,0,de63f78f36e74361b6c9993d12d8b7c8,0,,[],0,0,0,0
3,10062,5,,10047,Zabbix server: Number of processed character v...,"zabbix[wcache,values,str]",1m,1w,365d,0,...,0,0,b58fdd85cd6a48489d0b983ee8ebe97b,0,,[],0,0,0,0
4,10063,5,,10047,Zabbix server: Number of processed log values ...,"zabbix[wcache,values,log]",1m,1w,365d,0,...,0,0,dae76dc6c4ce4707be7cf9534efc0233,0,,[],0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7476,46991,18,,10501,sda: Disk write rate,vfs.dev.write.rate[sda],0,7d,365d,0,...,0,0,,0,,[],1703085669,905096709,6.048392837628446,5.759073005441059
7477,46992,18,,10501,sda: Disk utilization,vfs.dev.util[sda],0,7d,365d,0,...,0,0,,0,,[],1703085669,905096709,1.2030319819898339,1.058604170942345
7478,46993,18,,10501,sda: Disk read time (rate),vfs.dev.read.time.rate[sda],0,7d,365d,0,...,0,0,,0,,[],1703085669,905096709,0,0
7479,46994,18,,10501,sda: Disk read rate,vfs.dev.read.rate[sda],0,7d,365d,0,...,0,0,,0,,[],1703085669,905096709,0,0


## Marge host and item

In [106]:
#marge host and items
df_merge = df_host.merge(df_item, left_on='hostid', right_on='hostid')
df_merge = df_merge.rename(columns={"name_x": "host_name", "name_y": "item_name"})
df_merge

,hostid,host,host_name,itemid,type,snmp_oid,item_name,key_,delay,history,...,allow_traps,discover,uuid,state,error,parameters,lastclock,lastns,lastvalue,prevvalue
0,10439,360-Webserver-02,360-Webserver-02,36594,0,,Host name of Zabbix agent running,agent.hostname,1h,7d,...,0,0,,0,,[],1703009394,234507871,zabbix,
1,10439,360-Webserver-02,360-Webserver-02,36595,0,,Zabbix agent ping,agent.ping,1m,7d,...,0,0,,0,,[],1703085655,650350127,1,1
2,10439,360-Webserver-02,360-Webserver-02,36596,0,,Version of Zabbix agent running,agent.version,1h,7d,...,0,0,,0,,[],1703005796,167110658,5.4.1,
3,10439,360-Webserver-02,360-Webserver-02,36597,0,,Maximum number of open file descriptors,kernel.maxfiles,1h,7d,...,0,0,,0,,[],1703005797,182787181,9223372036854775807,0
4,10439,360-Webserver-02,360-Webserver-02,36598,0,,Maximum number of processes,kernel.maxproc,1h,7d,...,0,0,,0,,[],1703005798,199334621,4194304,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3629,10522,10.10.10.164,maholan-web-wordpress,46958,18,,sda: Disk average queue size (avgqu-sz),vfs.dev.queue_size[sda],0,7d,...,0,0,,0,,[],1703085632,192842011,0.0724291996005776,0.07385237377010086
3630,10522,10.10.10.164,maholan-web-wordpress,46959,0,,/: Free inodes in %,"vfs.fs.inode[/,pfree]",1m,7d,...,0,0,,0,,[],1703085639,290541176,97.401482,97.401482
3631,10522,10.10.10.164,maholan-web-wordpress,46960,0,,/: Space utilization,"vfs.fs.size[/,pused]",1m,7d,...,0,0,,0,,[],1703085640,311526246,9.000013,8.998083
3632,10522,10.10.10.164,maholan-web-wordpress,46961,0,,/: Total space,"vfs.fs.size[/,total]",1m,7d,...,0,0,,0,,[],1703085641,330151906,316203216896,316203216896


## Clean data

In [111]:
# cleasing
df_clean = df_merge[df_merge['item_name'].str.contains('cpu uti|memory uti|disk uti', flags=re.IGNORECASE, regex=True)]
df_clean =  df_clean[['hostid','host','host_name','itemid','item_name','lastclock','units','lastvalue','prevvalue','description']]

# change UNIXTIME to local datetime(UTC) and convert to local Bangkok time zone
df_clean['lastclock'] = pd.to_datetime(df_clean['lastclock'],unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Bangkok') 
df_clean

# lst_item_id = list(df_clean['itemid'])
# len(lst_item_id)

C:\Users\jdpsk\AppData\Local\Temp\ipykernel_8972\1464382845.py:6: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_clean['lastclock'] = pd.to_datetime(df_clean['lastclock'],unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Bangkok')


,hostid,host,host_name,itemid,item_name,lastclock,units,lastvalue,prevvalue,description
40,10439,360-Webserver-02,360-Webserver-02,36634,CPU utilization,2023-12-20 22:21:36+07:00,%,0.12296999999999514,0.1417110000000008,CPU utilization in %
41,10439,360-Webserver-02,360-Webserver-02,36635,Memory utilization,2023-12-20 22:19:28+07:00,%,10.591815999999994,10.592895999999996,Memory used percentage is calculated as (100-p...
55,10439,360-Webserver-02,360-Webserver-02,37003,sda: Disk utilization,2023-12-20 22:21:36+07:00,%,0.25949445472409266,0.1330581834799742,This item is the percentage of elapsed time th...
115,10440,360-Webserver-03,360-Webserver-03,36700,CPU utilization,2023-12-20 22:21:36+07:00,%,0.1750550000000004,0.17502199999999846,CPU utilization in %
116,10440,360-Webserver-03,360-Webserver-03,36701,Memory utilization,2023-12-20 22:21:36+07:00,%,11.348037000000005,11.344497000000004,Memory used percentage is calculated as (100-p...
...,...,...,...,...,...,...,...,...,...,...
3533,10521,10.10.10.169,proxy-server-238,46814,Memory utilization,2023-12-20 22:21:36+07:00,%,15.085701,15.061397,Memory used percentage is calculated as (100-p...
3555,10521,10.10.10.169,proxy-server-238,46926,sda: Disk utilization,2023-12-20 22:21:36+07:00,%,0.9306208592756797,2.6216130954823202,This item is the percentage of elapsed time th...
3603,10522,10.10.10.164,maholan-web-wordpress,46879,CPU utilization,2023-12-20 22:21:36+07:00,%,1.5319109999999938,1.4753759999999971,CPU utilization in %
3604,10522,10.10.10.164,maholan-web-wordpress,46880,Memory utilization,2023-12-20 22:21:36+07:00,%,26.360366,26.437790000000007,Memory used percentage is calculated as (100-p...


## useful dataframe

In [145]:
df_des =  df_clean[['hostid','host','host_name','itemid','item_name','units','description']]
df_des

,hostid,host,host_name,itemid,item_name,units,description
40,10439,360-Webserver-02,360-Webserver-02,36634,CPU utilization,%,CPU utilization in %
41,10439,360-Webserver-02,360-Webserver-02,36635,Memory utilization,%,Memory used percentage is calculated as (100-p...
55,10439,360-Webserver-02,360-Webserver-02,37003,sda: Disk utilization,%,This item is the percentage of elapsed time th...
115,10440,360-Webserver-03,360-Webserver-03,36700,CPU utilization,%,CPU utilization in %
116,10440,360-Webserver-03,360-Webserver-03,36701,Memory utilization,%,Memory used percentage is calculated as (100-p...
...,...,...,...,...,...,...,...
3533,10521,10.10.10.169,proxy-server-238,46814,Memory utilization,%,Memory used percentage is calculated as (100-p...
3555,10521,10.10.10.169,proxy-server-238,46926,sda: Disk utilization,%,This item is the percentage of elapsed time th...
3603,10522,10.10.10.164,maholan-web-wordpress,46879,CPU utilization,%,CPU utilization in %
3604,10522,10.10.10.164,maholan-web-wordpress,46880,Memory utilization,%,Memory used percentage is calculated as (100-p...


## Load histrory data 

In [135]:
zapi = ZabbixAPI("http://10.10.10.20/zabbix")

#start_date = datetime.datetime.today() + relativedelta(months=-1) #last one month from present
#start_date = datetime.timedelta(days=3) #last 30 days from present
start_date = datetime.datetime.strptime("2023-11-01 00:00:00", "%Y-%m-%d %H:%M:%S") # select by date
# end_date = datetime.datetime.today() 
end_date = datetime.datetime.strptime("2023-12-01 00:00:00", "%Y-%m-%d %H:%M:%S") # select by date

start_unix = time.mktime(start_date.timetuple()) #ช่วงเวลาเริ่ม 
end_unix = time.mktime(end_date.timetuple()) # ช่วงเวลาจบ
print(f"start_date : {start_date} >>  start_unix : {start_unix}")
print(f"end_date : {end_date} >> end_unix : {end_unix}")

df_htr = pd.DataFrame()
for i in lst_item_id:
        method = "trend.get" 
        params = {
                #"output": "extend",
                # "hostids" :10440,
                "itemids" : i ,	
                "time_from": int(start_unix),
                "time_till": int(end_unix),
                #"filter": {
                #    "itemid": [36634,36700]
                #},
                "sortfield": "clock",
                "sortorder": "DESC",
                "limit": 1000000
        }
        print(i)
        result = zapi.do_request(
        method,
        params=params,
        )
        

# print(result['result'])
# print(result_dict)

        df_tmp = pd.DataFrame(result['result'])
        df_htr = pd.concat([df_htr, df_tmp], ignore_index=True)
        # df_test = df_test.append(i, ignore_index=True)
        # df_test['clock'] = pd.to_datetime(df_test['clock'],unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Bangkok') 

df_htr

start_date : 2023-11-01 00:00:00 >>  start_unix : 1698771600.0
end_date : 2023-12-01 00:00:00 >> end_unix : 1701363600.0
36634
36635
37003
36700
36701
37028
36766
36767
37053
36832
36833
37086
36898
36899
37119
36964
36965
37152
39665
39666
39832
39839
39846
39853
39860
39867
39874
39881
39888
39895
39902
39909
39916
39923
39930
39937
39944
39951
39958
39965
39972
39979
39986
39993
39731
39732
40264
40271
40278
40285
40292
40299
40306
40313
40320
40327
40334
40341
40348
40355
40362
40369
40376
40383
40390
40397
40404
40411
40418
40425
40432
40161
40162
40737
40744
40751
40758
40765
40772
40779
40786
40793
40800
40807
40814
40821
40828
40835
40842
40849
40856
40863
40870
40877
40884
40891
40898
40905
44300
44301
44471
44366
44367
44492
44432
44433
44517
44569
44570
45004
44635
44636
45025
44701
44702
45046
44767
44768
45079
44833
44834
45100
44899
44900
45121
44965
44966
46992
45169
45170
45274
45235
45236
45299
45347
45348
45452
45413
45414
45473
45521
45522
45626
45587
45588
45678
456

,itemid,clock,num,value_min,value_avg,value_max
0,36634,1698771600,60,0.08752699999999436,0.13358373333333257,0.2063270000000017
1,36634,1698775200,60,0.08959799999999518,0.136291483333333,0.5168819999999954
2,36634,1698778800,60,0.08961700000000405,0.12423425000000017,0.16462799999999334
3,36634,1698782400,60,0.08751800000000287,0.12698086666666766,0.24382599999999854
4,36634,1698786000,60,0.09377499999999372,0.12795248333333284,0.19173499999999422
...,...,...,...,...,...,...
134101,46955,1701349200,60,0,0.8672807134044166,2.1917627837054794
134102,46955,1701352800,60,0.23913118495200938,1.0090674302863,5.59979024844435
134103,46955,1701356400,60,0.15289033701422272,1.0113775995300855,3.3711214502256848
134104,46955,1701360000,60,0.2709212067110488,1.5017011736265666,7.008774491531522


In [136]:
def avg_item(column, item):
    itm_avg = column.astype(float).mean()
    return item, itm_avg

In [143]:
tuple_avg = ()
for i in lst_item_id:
    df_name = f'df_{i}' 
    df_name = df_htr[df_htr['itemid']== str(i)]
    avg_itm = avg_item(df_name['value_avg'],i)  
    tuple_avg = tuple_avg + (avg_itm,)
len(tuple_avg)

190

In [152]:
df_avg = pd.DataFrame(tuple_avg, columns=['itemid', 'item_avg'])
df_avg

,itemid,item_avg
0,36634,0.134241
1,36635,10.434947
2,37003,0.277027
3,36700,0.225139
4,36701,11.223749
...,...,...
185,46814,15.637374
186,46926,2.386416
187,46879,0.862260
188,46880,22.730047


In [153]:
df_total_avg = df_des.merge(df_avg, left_on='itemid', right_on='itemid')
df_total_avg

,hostid,host,host_name,itemid,item_name,units,description,item_avg
0,10439,360-Webserver-02,360-Webserver-02,36634,CPU utilization,%,CPU utilization in %,0.134241
1,10439,360-Webserver-02,360-Webserver-02,36635,Memory utilization,%,Memory used percentage is calculated as (100-p...,10.434947
2,10439,360-Webserver-02,360-Webserver-02,37003,sda: Disk utilization,%,This item is the percentage of elapsed time th...,0.277027
3,10440,360-Webserver-03,360-Webserver-03,36700,CPU utilization,%,CPU utilization in %,0.225139
4,10440,360-Webserver-03,360-Webserver-03,36701,Memory utilization,%,Memory used percentage is calculated as (100-p...,11.223749
...,...,...,...,...,...,...,...,...
185,10521,10.10.10.169,proxy-server-238,46814,Memory utilization,%,Memory used percentage is calculated as (100-p...,15.637374
186,10521,10.10.10.169,proxy-server-238,46926,sda: Disk utilization,%,This item is the percentage of elapsed time th...,2.386416
187,10522,10.10.10.164,maholan-web-wordpress,46879,CPU utilization,%,CPU utilization in %,0.862260
188,10522,10.10.10.164,maholan-web-wordpress,46880,Memory utilization,%,Memory used percentage is calculated as (100-p...,22.730047


In [224]:
df_total_avg[df_total_avg['hostid']=='10479']

,hostid,host,host_name,itemid,item_name,units,description,item_avg
18,10479,10.10.10.14,Big Data Data node 1,39665,CPU utilization,%,CPU utilization in %,2.449366
19,10479,10.10.10.14,Big Data Data node 1,39666,Memory utilization,%,Memory used percentage is calculated as (100-p...,27.229107
20,10479,10.10.10.14,Big Data Data node 1,39832,sdq: Disk utilization,%,This item is the percentage of elapsed time th...,0.013679
21,10479,10.10.10.14,Big Data Data node 1,39839,sdp: Disk utilization,%,This item is the percentage of elapsed time th...,0.014090
22,10479,10.10.10.14,Big Data Data node 1,39846,sdo: Disk utilization,%,This item is the percentage of elapsed time th...,0.014535
23,10479,10.10.10.14,Big Data Data node 1,39853,sdt: Disk utilization,%,This item is the percentage of elapsed time th...,0.014540
24,10479,10.10.10.14,Big Data Data node 1,39860,sdr: Disk utilization,%,This item is the percentage of elapsed time th...,0.015555
25,10479,10.10.10.14,Big Data Data node 1,39867,sdu: Disk utilization,%,This item is the percentage of elapsed time th...,0.014651
26,10479,10.10.10.14,Big Data Data node 1,39874,sdv: Disk utilization,%,This item is the percentage of elapsed time th...,0.013973
27,10479,10.10.10.14,Big Data Data node 1,39881,sdw: Disk utilization,%,This item is the percentage of elapsed time th...,0.026678


In [220]:
def sum_item(column):
    disk_sum = column.astype(float).mean()
    return disk_sum

In [221]:
def for_each_host(host_id):
    df_name = df_total_avg[df_total_avg['hostid']==str(host_id)]
    df_name_cpu = df_name[df_name['item_name'].str.contains('CPU utilization', flags=re.IGNORECASE, regex=True)]
    df_name_mem = df_name[df_name['item_name'].str.contains('Memory utilization', flags=re.IGNORECASE, regex=True)]
    df_name_disk = df_name[df_name['item_name'].str.contains('Disk utilization', flags=re.IGNORECASE, regex=True)]

    item_avg_tuple = (df_name_cpu['item_avg'],df_name_mem['item_avg'],df_name_disk['item_avg'])
    values_list = [series.values[0] for series in item_avg_tuple]

    if len(df_name_disk) >1:
        values_list[2] = sum_item(df_name_disk['item_avg'])


    return print('monthly avg for hostid (%):', host_id,'cpu :', values_list[0], 'mem :', values_list[1], 'disk :', values_list[2], 'len disk', len(df_name_disk['item_avg']))

In [222]:
for_each_host(10479)

monthly avg for hostid (%): 10479 cpu : 2.4493664647711513 mem : 27.22910725182617 disk : 0.013884755212728701 len disk 24
